In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from urllib import request
import json
import random
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest



In [2]:
df_devices = pd.read_csv("all_devices_v1.csv.gz")
print(df_devices.shape)
print(df_devices.nunique())
df_devices.to_csv("all_devices_v1.csv.gz",compression="gzip",index=False)
df_devices.head()

(76261, 9)
Unnamed: 0                  76261
network_id                      4
device_id                   71663
type                            7
model                        1357
manufacturer                  263
operating_system               80
operating_system_version      266
id                          76261
dtype: int64


,Unnamed: 0,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version,id
0,113,0,1028623,MOBILE_PHONE,Galaxy S8,Samsung,Android,9,0_1028623
1,587,0,48047,MOBILE_PHONE,Galaxy Note 8,Samsung,Android,9,0_48047
2,668,0,123568,MOBILE_PHONE,H918,LG Electronics,Android,8.0.0,0_123568
3,830,0,95366,MOBILE_PHONE,iPhone 6,"Apple, Inc.",iOS,NaN,0_95366
4,886,0,1755023,TABLET,iPad,Apple,iOS,NaN,0_1755023


In [3]:
df_devices.isna().sum(axis=1)

0        0
1        0
2        0
3        1
4        1
        ..
76256    0
76257    0
76258    3
76259    0
76260    0
Length: 76261, dtype: int64

In [4]:
%%time
df = pd.read_csv("all_sessions_v1.csv.gz")

print(df.shape)
# print(df_devices.nunique())
df.tail()

(22463971, 44)
CPU times: user 1min 55s, sys: 26.2 s, total: 2min 21s
Wall time: 2min 26s


,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,id,row_nan_count,service_device_id_count,host_count,device_id_count,id-service_device_counts,port_dst_count
22463966,3,2701197,1565287200,96162,255197,443,TCP,NaN,19,2311,0,0.00000,1,NaN,1,19,0,2311,2311,2311.00000,NaN,NaN,0,0,0.00000,NaN,NaN,121.63158,121.63158,121.63158,121.63158,NaN,NaN,NaN,NaN,NaN,NaN,3_2701197,12,NaN,109945,309,309,13380015
22463967,3,2701266,1565283600,337758,347360,443,TCP,700571.0,58,1852,3906,0.00000,0,NaN,7,29,29,451,139,264.57143,NaN,156.99879,1353,135,558.00000,NaN,565.68395,69.50000,56.37500,66.41752,69.50000,5.27282,216.33333,67.5,108.22874,67.5,60.64558,3_2701266,3,2159.0,1148,501,501,13380015
22463968,3,2702227,1565287200,381770,329778,1146,TCP,2305795.0,114,2337,2964,0.00000,0,NaN,57,57,57,41,41,41.00000,NaN,0.00000,52,52,52.00000,NaN,0.00000,41.00000,41.00000,41.00000,41.00000,0.00000,52.00000,52.0,52.00000,52.0,0.00000,3_2702227,3,5.0,5,314,314,9
22463969,3,2703918,1565298000,377492,326062,53,UDP,NaN,14,535,849,NaN,0,NaN,7,7,7,103,58,76.42857,NaN,16.17464,168,80,121.28571,NaN,35.52330,103.00000,58.00000,76.42857,72.00000,16.17464,168.00000,80.0,121.28571,131.0,35.52330,3_2703918,5,NaN,62303,675,675,999553
22463970,3,2710107,1565287200,30734,222102,443,TCP,NaN,22,4039,0,0.16458,1,NaN,1,22,0,4039,4039,4039.00000,NaN,NaN,0,0,0.00000,NaN,NaN,183.59091,183.59091,183.59091,183.59091,NaN,NaN,NaN,NaN,NaN,NaN,3_2710107,12,NaN,419135,90,90,13380015


### datetime features
* cyclical datetime : https://www.kaggle.com/avanwyk/encoding-cyclical-features-for-deep-learning
* weekend, workhours

In [5]:
df["time"] = pd.to_datetime(df.timestamp,unit="s")

In [6]:
def cyclical_encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [7]:
# df['month'] = df.datetime.dt.month
# df = encode(df, 'month', 12)

df['dayOfWeek'] = df.time.dt.dayofweek # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.dayofweek.html#pandas.Series.dt.dayofweek

df = cyclical_encode(df, 'dayOfWeek', 7)

df['hour'] = df.time.dt.hour
df = cyclical_encode(df, 'hour', 24)

# Monday=0, Sunday=6.
df["work_day"] = (df.dayOfWeek>=6).astype(int)

df["work_hours"] = ((df.hour>=8) & (df.hour<=18)).astype(int)

In [8]:
# df["service_device_id_count"] = df.groupby("service_device_id")["packets_count"].transform("size")
# df["host_count"] = df.groupby("host")["packets_count"].transform("size")
# df["device_id_count"] = df.groupby("device_id")["packets_count"].transform("size")
# df["id-service_device_counts"] = df.groupby(["id","device_id"])["packets_count"].transform("size")
# df["port_dst_count"] = df.groupby("port_dst")["packets_count"].transform("size")

In [9]:
# df.to_csv("all_sessions_v1.csv.gz",compression="gzip",index=False)

# Baseline

# basic features + sub
* base on baseline

* features per network
* orig: As we mentioned before we are going to select only 5 features: unique hosts count, unique port count, total packets transferred, total inbound bytes and total outbound bytes. Lets create the feature set for each network

In [10]:
list(df.columns)

['network_id',
 'device_id',
 'timestamp',
 'host',
 'host_ip',
 'port_dst',
 'transport_protocol',
 'service_device_id',
 'packets_count',
 'outbound_bytes_count',
 'inbound_bytes_count',
 'packet_loss',
 'retransmit_count',
 'latency',
 'session_count',
 'outbound_packets_count',
 'inbound_packets_count',
 'outbound_bytes_max',
 'outbound_bytes_min',
 'outbound_bytes_mean',
 'outbound_bytes_median',
 'outbound_bytes_stddev',
 'inbound_bytes_max',
 'inbound_bytes_min',
 'inbound_bytes_mean',
 'inbound_bytes_median',
 'inbound_bytes_stddev',
 'outbound_packet_size_max',
 'outbound_packet_size_min',
 'outbound_packet_size_mean',
 'outbound_packet_size_median',
 'outbound_packet_size_stddev',
 'inbound_packet_size_max',
 'inbound_packet_size_min',
 'inbound_packet_size_mean',
 'inbound_packet_size_median',
 'inbound_packet_size_stddev',
 'id',
 'row_nan_count',
 'service_device_id_count',
 'host_count',
 'device_id_count',
 'id-service_device_counts',
 'port_dst_count',
 'time',
 'dayOfW

In [11]:
df.head()


,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,id,row_nan_count,service_device_id_count,host_count,device_id_count,id-service_device_counts,port_dst_count,time,dayOfWeek,dayOfWeek_sin,dayOfWeek_cos,hour,hour_sin,hour_cos,work_day,work_hours
0,0,35,1565074800,413484,357105,49152,TCP,790889.0,260,21855,7815,0.0675,67,NaN,39,235,25,930,465,560.38462,NaN,190.21935,1563,0,200.38462,NaN,529.37001,93.00000,93.00000,93.00000,93.00000,0.00000,312.6,312.6,312.6,312.6,0.0,0_35,3,36.0,52,7539,7539,29303,2019-08-06 07:00:00,1,0.78183,0.62349,7,0.96593,-0.25882,0,0
1,0,35,1565053200,252449,218408,49153,TCP,1604622.0,178,8326,28656,0.0000,16,NaN,18,88,90,467,427,462.55556,467.0,12.93523,1592,1592,1592.00000,1592.0,0.00000,106.75000,93.40000,94.88333,93.00000,4.31713,318.4,318.4,318.4,318.0,0.0,0_35,1,28.0,53,7539,7539,12926,2019-08-06 01:00:00,1,0.78183,0.62349,1,0.25882,0.96593,0,0
2,0,57,1565082000,393482,319956,443,TCP,NaN,67,4013,0,0.0000,40,NaN,7,67,0,3521,82,573.28571,NaN,1299.81982,0,0,0.00000,NaN,0.00000,64.01818,41.00000,44.28831,41.00000,8.70005,NaN,NaN,NaN,NaN,NaN,0_57,9,NaN,19752,4897,4897,13380015,2019-08-06 09:00:00,1,0.78183,0.62349,9,0.70711,-0.70711,0,1
3,0,57,1565082000,46068,223744,443,TCP,NaN,45,9158,0,0.0000,3,NaN,4,45,0,3401,935,2289.50000,2411.0,1018.07449,0,0,0.00000,0.0,0.00000,226.73333,133.57143,194.96824,209.00000,41.77727,NaN,NaN,NaN,NaN,NaN,0_57,7,NaN,17209,4897,4897,13380015,2019-08-06 09:00:00,1,0.78183,0.62349,9,0.70711,-0.70711,0,1
4,0,57,1565082000,389390,113695,443,TCP,NaN,9,1671,0,0.0000,0,NaN,1,9,0,1671,1671,1671.00000,NaN,NaN,0,0,0.00000,NaN,NaN,185.66667,185.66667,185.66667,185.66667,NaN,NaN,NaN,NaN,NaN,NaN,0_57,12,NaN,16114,4897,4897,13380015,2019-08-06 09:00:00,1,0.78183,0.62349,9,0.70711,-0.70711,0,1


In [13]:
# print(df.nunique())

In [14]:
df.shape

(22463971, 53)

In [37]:
def extract_all_features(df):
    return df.drop("network_id",axis=1).groupby(["device_id"]).aggregate(["min","max","std","nunique","sum","last","mean"])

#     return df.drop("network_id",axis=1).groupby(["device_id"]).aggregate(["max","mean"])



In [25]:
def extract_features(df):
    return df.groupby(["device_id"]).aggregate({"host": "nunique",
                                                "port_dst": "nunique",
                                                 'row_nan_count':["min","max","std"],
                                                "service_device_id": "nunique",
 'service_device_id_count':["min","max"],
 'host_count':["min","max"],
 'device_id_count':["min","max"],
 'id-service_device_counts':["min","max"],                                
 'port_dst_count':["min","max","mean"],
                                                "row_nan_count":["min","max","mean"],
                                                
                                                "host_ip": "nunique",
                                                "packets_count":["sum","mean"], "inbound_bytes_count": ["sum","mean"], 
                                                "outbound_bytes_count": ["sum","mean"],
                                               "latency":["max","min","std","mean"],
                                               
   'dayOfWeek':["mean","std","nunique"],
 'dayOfWeek_sin':["mean","std","nunique"],
 'dayOfWeek_cos':["mean","std","nunique"],
 'hour':["mean","std","nunique"],
 'hour_sin':["mean","std","max"],
 'hour_cos':["mean","std","max"],
 'work_day':["mean","std","max"],
 'work_hours':["mean","std","max"],                                             
                                               })

In [26]:
%%time

networks_fs = []


# for i in range(4):
#     print(i)
# #     networks_fs.append(extract_features(df[df.network_id == i])) # selected fetas
#     networks_fs.append(extract_all_features(df[df.network_id == i])) # all feats
    
#     ### fill nans
# for i in range(4):
# #     networks_fs[i].fillna(-1,inplace=True)
#     networks_fs[i].dropna(axis=1,inplace=True)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 21.9 µs



### Modeling Fun
* As we've mentioned we used the Elliptic Envelope model for our anomaly detection. We use the decision_function in order to receive the confidence and we normalize the values to values between 0 and 1.

In [29]:
# We use the simple min-max normalization in order to normalize the confidence values to 0-1 range.
# Higher score means that this device is probably more anomalous.
def calc_normalized_decision(decision_function_result):
    decision_function_result = -1 * decision_function_result
    minimum = decision_function_result.min()
    maximum = decision_function_result.max()
    return (decision_function_result - minimum) / (maximum - minimum)

def detect_anomalies(feature_set, param = 0.05, bootstrap=False, n_estimators=120,
                     behaviour='new',max_samples=700, n_jobs=3):
    #   ee = EllipticEnvelope(contamination=param).fit(feature_set.values)
    ee = IsolationForest(contamination=param,n_estimators=n_estimators,bootstrap=bootstrap,max_samples=max_samples, behaviour=behaviour,n_jobs=n_jobs).fit(feature_set.values)
    decision_function_result = ee.decision_function(feature_set.values)
    feature_set["confidence"] = calc_normalized_decision(decision_function_result)
    return feature_set

In [28]:
# networks_fs = []

# param = 0.05
# bootstrap = True
# n_estimators = 80
# behaviour = "new"

# for i in range(4):
#     print(i)
# #             networks_fs.append(extract_features(df[df.network_id == i])
#     networks_fs.append(extract_all_features(df[df.network_id == i]))
# ### drop or fill nans
# for i in range(4):
# #     networks_fs[i].fillna(-1,inplace=True)
#     networks_fs[i].dropna(axis=1,inplace=True)

# for i in range(4):
#     networks_fs[i] = detect_anomalies(feature_set=networks_fs[i], param=param,  bootstrap=bootstrap,
#                                       n_estimators=n_estimators, behaviour=behaviour) # ORIG
# #         networks_fs[i] = detect_anomalies(feature_set=networks_fs[i])

# # Remove the device_id from index and Add network id to each data set
# for i in range(4):
#     networks_fs[i].reset_index(level=0, inplace=True)
#     networks_fs[i]["network_id"] = i 

# df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
# df_to_submit.fillna(0,inplace=True) # handle nans
# df_to_submit.head()
# arr_to_submit = df_to_submit.to_json(orient='values')


# leaderboard_name = "armis"
# host = "leaderboard.datahack.org.il"

# # Name of the user
# submitter = ".303Overfitter"

# predictions = json.loads(arr_to_submit)

# jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
# data = jsonStr.encode('utf-8')
# req = request.Request(f"https://{host}/{leaderboard_name}/api/",
#                       headers={'Content-Type': 'application/json'},
#                       data=data)
# resp = request.urlopen(req)

# ret = json.load(resp)

# print(ret['score'])

0
1
2
3


,network_id,device_id,confidence
,,,
0,0,33,0.05768
1,0,35,0.35974
2,0,40,0.22719
3,0,41,0.09512
4,0,53,0.25350


In [46]:
# for i in range(4):
#     networks_fs[i].drop("index",axis=1,inplace=True)
#     networks_fs[i].drop('level_0',axis=1,inplace=True)
   
# networks_fs[0].tail()

/Users/danofer/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


device_id   timestamp                 host               host_ip              port_dst             packets_count             outbound_bytes_count  \
                        max         mean     max          mean     max         mean      max        mean           max        mean                  max   
32385   1769648  1565391600  1.56539e+09  270100  223983.00000  338408  249141.0000      443   183.00000            21    11.33333                  793   
32386   1769699  1565391600  1.56539e+09  377835  232329.75000  331715  201478.8750     5223   911.75000            27    10.87500                 6576   
32387   1769744  1565391600  1.56539e+09   42450   42450.00000   36868   36868.0000      443   443.00000             6     6.00000                  270   
32388   1769760  1565391600  1.56539e+09  202397  109867.00000  175513   95331.0000    19305  8195.33333          6374  2129.33333              2684631   
32389   1769802  1565391600  1.56539e+09  446002  238192.99213  380172  192121.4252      993   413.49606          8912   254.80315               111635   

                    inbound_bytes_count               retransmit_count          session_count          outbound_packets_count              \
               mean                 max          mean              max     mean           max     mean                    max        mean   
32385     407.66667                1316     698.33333                1  0.33333             1  1.00000                     13     6.66667   
32386    1418.50000                6287    1657.87500                6  1.75000             2  1.12500                     15     6.00000   
32387     270.00000                  80      80.00000                0  0.00000             1  1.00000                      4     4.00000   
32388  895017.66667              257093   87642.33333                0  0.00000             1  1.00000                   4329  1445.33333   
32389   10535.09449            10412893  217743.20472              156  9.21260             5  1.54331                   1555    83.97638   

      inbound_packets_count            outbound_bytes_max               outbound_bytes_min               outbound_bytes_mean               inbound_bytes_max  \
                        max       mean                max          mean                max          mean                 max          mean               max   
32385                     8    4.66667                793     407.66667                793     407.66667               793.0     407.66667              1316   
32386                    12    4.87500               6403    1396.87500               3568     618.12500              3568.0    1007.50000              6287   
32387                     2    2.00000                270     270.00000                270     270.00000               270.0     270.00000                80   
32388                  2045  684.00000            2684631  895017.66667            2684631  895017.66667           2684631.0  895017.66667            257093   
32389                  7357  170.82677              84050    9414.40945              83338    5733.87402             83338.0    7276.71260          10412585   

                    inbound_bytes_min              inbound_bytes_mean               outbound_packet_size_max            outbound_packet_size_min             \
               mean               max         mean                max          mean                      max       mean                      max       mean   
32385     698.33333              1316    698.33333             1316.0     698.33333                 64.00000   61.80000                 64.00000   61.80000   
32386    1644.75000              6287   1592.50000             6287.0    1618.62500                711.44444  163.55972                237.86667   85.44167   
32387      80.00000                80     80.00000               80.0      80.00000                 67.50000   67.50000                 67.50000   67.50000   
32388   87642.33

In [53]:
def by_force(param, bootstrap, n_estimators, behaviour,networks_fs = []):
    if networks_fs == []:

        for i in range(4):
            print(i)
#             networks_fs.append(extract_features(df[df.network_id == i])
            networks_fs.append(extract_all_features(df[df.network_id == i]))
        ### drop or fill nans
        for i in range(4):
        #     networks_fs[i].fillna(-1,inplace=True)
            networks_fs[i].dropna(axis=1,inplace=True)

    for i in range(4):
        try: networks_fs[i].drop("index",axis=1,inplace=True)
        except:pass
        try: networks_fs[i].drop('level_0',axis=1,inplace=True)
        except:pass
   
        
#     for i in range(4):
        networks_fs[i] = detect_anomalies(feature_set=networks_fs[i], param=param,
                                          bootstrap=bootstrap, n_estimators=n_estimators, behaviour=behaviour) # ORIG
#         networks_fs[i] = detect_anomalies(feature_set=networks_fs[i])
    
    # Remove the device_id from index and Add network id to each data set
    for i in range(4):
#         networks_fs[i].fillna(0,inplace=True)
        networks_fs[i].reset_index(level=0, inplace=True,)
        networks_fs[i]["network_id"] = i 
        
    df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
    df_to_submit.fillna(0,inplace=True) # handle nans
    df_to_submit.head()
    arr_to_submit = df_to_submit.to_json(orient='values')


    leaderboard_name = "armis"
    host = "leaderboard.datahack.org.il"

    # Name of the user
    submitter = ".303Overfitter"

    predictions = json.loads(arr_to_submit)

    jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
    data = jsonStr.encode('utf-8')
    req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                          headers={'Content-Type': 'application/json'},
                          data=data)
    resp = request.urlopen(req)
    
    ret = json.load(resp)
    return ret['score']


max_score = 0
max_param = 0.05

for i in [0.02,0.05, 0.08]:  
# while True:
# for j in range(3):
    
    i = random.random()
#     behaviour = random.choice(['old', 'new'])
    behaviour = "new"
    n_estimators = random.randint(40, 150)
    bootstrap = random.choice([True, False])
    score = by_force(i,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)
    print(score)
    if score > max_score:
        max_score = score
        max_param = i 
        max_n_estimators = n_estimators
        max_boostrap = bootstrap
    else:   
        print('resubmitting')
        score = by_force(i,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)        
        
        

0.7278395812101099


/Users/danofer/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


0.715401763347142
resubmitting


/Users/danofer/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)
/Users/danofer/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


0.730563370713096


### feat stats
* subtract features , get stats of perf

In [54]:
networks_fs[0].head()

index device_id   timestamp                 host               host_ip               port_dst              packets_count             outbound_bytes_count  \
                          max         mean     max          mean     max          mean      max         mean           max        mean                  max   
0     0        33  1565391600  1.56519e+09  226390  217906.28319  196053  188705.74336     1900    215.96460            16     2.97345                 3232   
1     1        35  1565391600  1.56519e+09  445250  204162.99629  384388  177976.09603    60945  44171.78445          1020   208.34899                70984   
2     2        40  1565391600  1.56520e+09  446768  203363.26984  385843  192765.70083    64287   2785.90131       3888627  1542.16908           1966323805   
3     3        41  1565391600  1.56519e+09  226390  135415.12245  196053  117264.91837     1900    974.05102            56    13.05102                11312   
4     4        53  1565078400  1.56508e+09  165322  165322.00000  143379  143379.00000      137    137.00000             2     2.00000                  156   

                inbound_bytes_count              retransmit_count           session_count           outbound_packets_count            inbound_packets_count  \
           mean                 max         mean              max      mean           max      mean                    max       mean                   max   
0     664.67257                   0  0.00000e+00                0   0.00000             5   2.46018                     16    2.97345                     0   
1   10137.60088              150484  3.07489e+04              513  27.84375            90  19.29633                    760  109.48534                   466   
2  690960.71049          2717883053  1.10823e+06             1537   9.00897            81   3.78778                1226721  481.97378               2661906   
3    2670.18367                   0  0.00000e+00                0   0.00000             7   2.90816                     56   13.05102                     0   
4     156.00000                   0  0.00000e+00                0   0.00000             1   1.00000                      2    2.00000                     0   

              outbound_bytes_max              outbound_bytes_min             outbound_bytes_mean              inbound_bytes_max                \
         mean                max         mean                max        mean                 max         mean               max          mean   
0     0.00000               1616    290.37168               1616   272.49558         1.61600e+03    282.02950                 0       0.00000   
1    98.86364               7876    722.68152               7876   503.55750         7.87600e+03    543.97892             60341    2225.79639   
2  1060.19531           91512765  39825.25017             296789  2824.64872         2.55367e+07  12954.61515         159505049  191783.02208   
3     0.00000               1616    879.12245               1616   758.46939         1.61600e+03    835.86774                 0       0.00000   
4     0.00000                156    156.00000                156   156.00000         1.56000e+02    156.00000                 0       0.00000   

  inbound_bytes_min              inbound_bytes_mean             outbound_packet_size_max            outbound_packet_size_min             \
                max         mean                max        mean                      max       mean                      max       mean   
0                 0      0.00000        0.00000e+00      0.0000                229.00000  227.80531                229.00000  227.80531   
1              4172   1449.74705        3.37195e+04   1577.2091                594.75000   99.13436                594.75000   93.60347   
2          28833300  29305.12974        3.52972e+07  66287.2364               3039.71429  306.59834               3039.71429  205.54393   
3                 0      0.00000        0.00000e+00      0.0000

In [67]:
networks_fs[0].reset_index(level=0).columns

MultiIndex([(                    'level_0',     ''),
            (                      'index',     ''),
            (                  'device_id',     ''),
            (                  'timestamp',  'max'),
            (                  'timestamp', 'mean'),
            (                       'host',  'max'),
            (                       'host', 'mean'),
            (                    'host_ip',  'max'),
            (                    'host_ip', 'mean'),
            (                   'port_dst',  'max'),
            (                   'port_dst', 'mean'),
            (              'packets_count',  'max'),
            (              'packets_count', 'mean'),
            (       'outbound_bytes_count',  'max'),
            (       'outbound_bytes_count', 'mean'),
            (        'inbound_bytes_count',  'max'),
            (        'inbound_bytes_count', 'mean'),
            (           'retransmit_count',  'max'),
            (           'retransmit_count', 'm

In [ ]:
# def select_features_overfit(networks_fs):
feat_stats = {}
# col_counts = networks_fs[0].shape[1] -1 # ignore first col of device id
f_names = list(networks_fs[0].columns).remove("device_id")

#baseline best score
baseline_best = 60
for f in range(5): # try only a few cols to speedup col_counts//120
    print(f)
    f_name = networks_fs[0].columns[f][0]
    print("col name",f_name)

    for i in range(4):
        try: networks_fs[i].drop("index",axis=1,inplace=True)
        except:pass
        try: networks_fs[i].drop('level_0',axis=1,inplace=True)
        except:pass
        
        networks_fs[i] = detect_anomalies(feature_set=networks_fs[i].drop([f_name],axis=1), param=param,  bootstrap=bootstrap, n_estimators=n_estimators, behaviour=behaviour) # ORIG


    # Remove the device_id from index and Add network id to each data set
    for i in range(4):  # DOES this fuck up feature index order ???? 
#         networks_fs[i].fillna(0,inplace=True)  # 
        networks_fs[i].reset_index(level=0, inplace=True)
        networks_fs[i]["network_id"] = i 

    df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
    df_to_submit.fillna(0,inplace=True)
    df_to_submit.head()
    arr_to_submit = df_to_submit.to_json(orient='values')


    leaderboard_name = "armis"
    host = "leaderboard.datahack.org.il"

    # Name of the user
    submitter = ".303BookwormOverfitter"

    predictions = json.loads(arr_to_submit)

    jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
    data = jsonStr.encode('utf-8')
    req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                          headers={'Content-Type': 'application/json'},
                          data=data)
    resp = request.urlopen(req)

    ret = json.load(resp)
    print("score",ret['score'])

    if  ret['score']>baseline_best:
        print("\n best score surpassed! \n")
    # save value to dict

#     feat_stats[networks_fs[0].columns[f][0]] = ret['score']

    feat_stats[f_name]= ret['score']
    
#     return(feat_stats)
        
        
        
        
        
# select_features_overfit(networks_fs)

In [ ]:
for i in range(4):
    networks_fs[i].drop(["level_0","index"],inplace=True,axis=1)

In [40]:
def by_force(param, bootstrap, n_estimators, behaviour,networks_fs = []):
    if networks_fs == []:

        for i in range(4):
            print(i)
            networks_fs.append(extract_features(df[df.network_id == i]))
        ### drop or fill nans
        for i in range(4):
        #     networks_fs[i].fillna(-1,inplace=True)
            networks_fs[i].dropna(axis=1,inplace=True)
        
        
    for i in range(4):
        networks_fs[i] = detect_anomalies(feature_set=networks_fs[i], param=param,  bootstrap=bootstrap, n_estimators=n_estimators, behaviour=behaviour) # ORIG
#         networks_fs[i] = detect_anomalies(feature_set=networks_fs[i])
    
    # Remove the device_id from index and Add network id to each data set
    for i in range(4):
        networks_fs[i].reset_index(level=0, inplace=True)
        networks_fs[i]["network_id"] = i 
        
    df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
    df_to_submit.fillna(0,inplace=True)
    df_to_submit.head()
    arr_to_submit = df_to_submit.to_json(orient='values')


    leaderboard_name = "armis"
    host = "leaderboard.datahack.org.il"

    # Name of the user
    submitter = "Overfitters"

    predictions = json.loads(arr_to_submit)

    jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
    data = jsonStr.encode('utf-8')
    req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                          headers={'Content-Type': 'application/json'},
                          data=data)
    resp = request.urlopen(req)
    
    ret = json.load(resp)
    return ret['score']



max_score = 0.5
max_param = 0.05
for i in [0.03,0.05, 0.09]:  

# while True:
# for j in range(2):
    
    i = random.random()
#     behaviour = random.choice(['old', 'new'])
    behaviour = "new"
    n_estimators = random.randint(50, 250)
    bootstrap = random.choice([True, False])
    score = by_force(i,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)
    print(score)
    if score > max_score:
        max_score = score
        max_param = i 
        max_n_estimators = n_estimators
        max_boostrap = bootstrap
    else:   
        print('resubmitting')
        score = by_force(i,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)        
        

URLError: <urlopen error [Errno 32] Broken pipe>

In [ ]:
by_force(1,  bootstrap, n_estimators, behaviour,networks_fs=networks_fs)

In [ ]:

# for i in range(4):
#     networks_fs[i] = detect_anomalies(feature_set=networks_fs[i], param=param,  bootstrap=bootstrap, n_estimators=n_estimators, behaviour=behaviour) # ORIG

# Remove the device_id from index and Add network id to each data set
# for i in range(4):
#     networks_fs[i].reset_index(level=0, inplace=True)
#     networks_fs[i]["network_id"] = i ## bug??
    
    
df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
df_to_submit.head()



## Submissions
* In order to update the Leader Board you need to send a POST request to following url: "https://leaderboard.datahack.org.il/armis/api/". Our Leader Board receives your anomalies results in the json form of [["network_id", "device_id", "confidence"]] - The order is important!

In [ ]:
arr_to_submit = df_to_submit.to_json(orient='values')

In [ ]:
df_to_submit.to_csv("sub_83.csv",index=False)

In [ ]:
json.dump(arr_to_submit, open('sub.json', 'w'))

In [ ]:
from urllib import request
import json

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "Overfitters"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))

In [ ]:
df_to_submit.to_json(orient='values')